## 1. Importar librerías necesarias

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Cargar modelo pre-entrenado

Empezaremos usando un modelo pre-entrenado en ImageNet (ResNet50)

In [ ]:
from src.models.vision_model import VisionModel

# Crear modelo
model = VisionModel(num_classes=1000, pretrained=True)
model.eval()

# Mover a GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print("Modelo cargado exitosamente!")

## 3. Probar el modelo con una imagen

Vamos a clasificar una imagen de ejemplo

In [ ]:
# Cargar labels de ImageNet
import requests

LABELS_URL = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
labels = requests.get(LABELS_URL).json()

# Transformaciones para la imagen
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image(image_path):
    """Predecir la clase de una imagen"""
    # Cargar y preprocesar imagen
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0).to(device)
    
    # Hacer predicción
    with torch.no_grad():
        output = model(img_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
    
    # Top 5 predicciones
    top5_prob, top5_idx = torch.topk(probabilities, 5)
    
    # Mostrar imagen
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Imagen de entrada')
    plt.show()
    
    # Mostrar predicciones
    print("\nTop 5 predicciones:")
    for i, (prob, idx) in enumerate(zip(top5_prob, top5_idx)):
        print(f"{i+1}. {labels[idx]}: {prob.item()*100:.2f}%")

# Ejemplo de uso (reemplaza con la ruta de tu imagen)
# predict_image('ruta/a/tu/imagen.jpg')

## 4. Preparar tu propio dataset

Para entrenar con tus propias imágenes, organiza tu dataset así:

```
data/
  train/
    clase1/
      img1.jpg
      img2.jpg
    clase2/
      img1.jpg
      img2.jpg
  val/
    clase1/
      img1.jpg
    clase2/
      img1.jpg
```

In [ ]:
from src.data.data_loader import create_dataloaders

# Crear dataloaders (cuando tengas datos)
# train_loader, val_loader = create_dataloaders('data', batch_size=32)
# print(f"Batches de entrenamiento: {len(train_loader)}")
# print(f"Batches de validación: {len(val_loader)}")

## 5. Siguiente paso: Entrenar tu modelo

Una vez que tengas tu dataset listo, puedes:

1. Fine-tunar el modelo pre-entrenado con tus datos
2. Entrenar desde cero con tu arquitectura personalizada
3. Usar transfer learning para acelerar el entrenamiento

Revisa el notebook `02_training.ipynb` para aprender a entrenar modelos.

## Recursos adicionales

- **PyTorch Tutorials**: https://pytorch.org/tutorials/
- **Fast.ai**: https://www.fast.ai/ (excelente para empezar)
- **Papers with Code**: https://paperswithcode.com/ (últimas investigaciones)
- **Hugging Face**: https://huggingface.co/ (modelos pre-entrenados)
- **Meta AI Research**: https://ai.meta.com/research/